In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
%%time
from modules.scouter_handler import ScouterHandler
from modules.scouter.config import default_newspaper_idx, default_graph_idx, default_timeline_idx
from modules.doc_graph_analyzer import DocGraphAnalyzer
from modules.multi_doc_summary import MultiDocSummary
from modules.timeline_summary import TimelineSummary
from modules.image_retrieve import ImageRetriever

from modules.utils import make_morp_sentence_list

Loading model parameters.
IO line:45 get_fileds, datatype text
model line:127 self.no_pack_padded_seq False
ModelConstructor line:191, model.obj_f
ml
ModelConstructor line:195, tgt vocab len 50004
ModelConstructor line:196, tgt vocab freq len 177439
Loading model parameters.
CPU times: user 5.84 s, sys: 1.68 s, total: 7.51 s
Wall time: 8.66 s


/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [9]:
keyword = "비트코인"
out_path = "results/"

min_rel_score = 2.0
max_length = 3000

In [10]:
scouter = ScouterHandler(size=1000)

In [11]:
%%time
query_body = scouter.make_keyword_query_body(keyword, filters=['news_id', 'node', 'edge', 'extContent'])
doc_info_list = scouter.search(query_body, data_type="graphs", trim_lower=True)

doc_info_list = [d for d in doc_info_list if d['rel_score'] > min_rel_score and len(d['extContent']) < max_length]
num_doc = len(doc_info_list)
num_doc

Query : 비트코인
Scroll idx : 1 (237 docs)
Trim lower 70% of docs (71 docs are cut)
Total retrieved Doc # :  165

CPU times: user 96 ms, sys: 16 ms, total: 112 ms
Wall time: 212 ms


In [12]:
%%time
doc_graph_anal = DocGraphAnalyzer(keyword, out_path)
doc_graph_anal.process_data(doc_info_list, top_doc_num=5, top_keyword_num=5)

Doc #  : 151
Node # : 1438
Edge # : 51847

Community 0 :  275 Keyword, (기술/블록체/세계/블록/체인)
Community 1 :  265 Keyword, (정부/규제/금융/은행/미국)
Community 2 :  239 Keyword, (투자/가격/시장/지난/시세)
Community 3 :  160 Keyword, (결제/업체/확인/시스템/사용)
Community 4 :   45 Keyword, (방송/압박/지지/조사/올림픽)
Community 5 :  208 Keyword, (경찰/설명/이용/채굴/자금)
Community 6 :  181 Keyword, (비트/가능/비트코인/수익/몰수)
Community 7 :   65 Keyword, (장관/법무/폐쇄/발언/부처)

CPU times: user 508 ms, sys: 0 ns, total: 508 ms
Wall time: 508 ms


In [13]:
%%time
top_doc_id_query_list = [scouter.make_doc_id_query_body(top_doc_ids, filters=["news_id", "newsTitle", "extContent", "analyzed_text"])
                         for top_doc_ids in doc_graph_anal.top_doc_id_list]
top_doc_info_list = [scouter.search(doc_id_query, data_type=default_newspaper_idx, trim_lower=False, silence=True)
                     for doc_id_query in top_doc_id_query_list]

top_doc_morph_sentence_list = [sum([make_morp_sentence_list(doc_info['analyzed_text'])
                                    for doc_info in doc_info_list], []) for doc_info_list in top_doc_info_list]
top_keyword_list = doc_graph_anal.top_keyword_list

CPU times: user 500 ms, sys: 28 ms, total: 528 ms
Wall time: 970 ms


In [ ]:
%%time
multi_doc_summ = MultiDocSummary(keyword, out_path)
multi_doc_summ.process_data(top_doc_morph_sentence_list, top_keyword_list)

In [9]:
%%time
query_body = scouter.make_keyword_query_body(keyword)
doc_info_list = scouter.search(query_body, data_type="summary_by_time", max_num_doc=1000, trim_lower=True)

Query : 미투
Scroll idx : 1 (813 docs)
Trim lower 70% of docs (243 docs are cut)
Total retrieved Doc # :  569

CPU times: user 1.01 s, sys: 212 ms, total: 1.22 s
Wall time: 1.87 s


In [10]:
%%time
time_line_summ = TimelineSummary(keyword, out_path)
time_line_summ.process_data(doc_info_list)

A :  0.010506391525268555
B :  0.9801366329193115
C :  0.0604701042175293
len(files) :  190
len(sents) :  15
len(sents) :  10
len(sents) :  19
len(sents) :  12
len(sents) :  12
len(sents) :  14
len(sents) :  13
len(sents) :  10
len(sents) :  16
len(sents) :  7
len(sents) :  10
len(sents) :  6
len(sents) :  7
len(sents) :  5
len(sents) :  1
len(sents) :  3
len(sents) :  13
len(sents) :  5
len(sents) :  2
len(sents) :  1
len(sents) :  1
len(sents) :  1
C-sub :  0.6147806644439697
D :  0.02740788459777832
CPU times: user 1.69 s, sys: 20 ms, total: 1.71 s
Wall time: 1.7 s


In [14]:
%%time
image_ret = ImageRetriever(keyword, out_path, from_google=True)
a = image_ret.process_data(doc_graph_anal.top_keyword_list)


Item no.: 1 --> Item name = 비트코인 정부 규제 금융 은행 미국
Item no.: 1 --> Item name = 비트코인 비트 가능 비트코인 수익 몰수
Evaluating...

Item no.: 1 --> Item name = 비트코인 방송 압박 지지 조사 올림픽

Item no.: 1 --> Item name = 비트코인 기술 블록체 세계 블록 체인

Item no.: 1 --> Item name = 비트코인 장관 법무 폐쇄 발언 부처
Item no.: 1 --> Item name = 비트코인 경찰 설명 이용 채굴 자금Evaluating...

Evaluating...

Evaluating...

Item no.: 1 --> Item name = 비트코인 결제 업체 확인 시스템 사용
Evaluating...


Item no.: 1 --> Item name = 비트코인 투자 가격 시장 지난 시세
Evaluating...
Evaluating...
Evaluating...
Starting Download...
Starting Download...
Starting Download...
Starting Download...
Starting Download...
Starting Download...
Starting Download...
Completed Image ====> 1. f6844a27e3a3c9fb40e77d6ea0db6946.jpg
Completed Image ====> 1. ps17092400316.jpg
Completed Image ====> 2. 2018020201011_0.jpg
Completed Image ====> 1. fd8d13e3-2489-472c-950a-fe804622f910.jpg
Starting Download...
Completed Image ====> 1. image_2112881331525605696709.jpg
Completed Image ====> 1. 01.15405772.1.jpg
Comple

Completed Image ====> 20. nisi20160912_0012171017_web.jpg

Errors: 0

Completed Image ====> 19. image_readtop_2018_696320_15415380623522327.jpg
can not fine image - continue to next image
Completed Image ====> 9. pro-bitcoin-ohio-bill-promotes-government-adoption-of-blockchain-ccn.jpg
can not fine image - continue to next image
Completed Image ====> 10. 2019020701435838.jpg
Completed Image ====> 11. 1499661599-35.jpg
Completed Image ====> 20. ssi_20180111174120_v.jpg

Errors: 1

Completed Image ====> 16. 0128154862889520190135.jpg
Completed Image ====> 12. coinclip_thread_8459062.7028189231550127853612.png
Completed Image ====> 17. 201712050480295258_1.jpg
can not fine image - continue to next image
Completed Image ====> 13. %ed%81%ac%eb%a1%9c%ec%8a%a4%ec%b2%b4%ec%9d%b8%ea%b8%b0%ec%88%a0%ea%b0%9c%eb%85%90%eb%8f%84.jpg
can not fine image - continue to next image
Completed Image ====> 17. %eb%a9%94%ec%9d%b8.jpg
Completed Image ====> 14. 1981902326_1456207779.42026.jpg
Completed Image ===

IndexError: list index out of range